In [1]:
import sys
from pathlib import Path

def in_colab():
    try:
        import google.colab  # noqa: F401
        return True
    except Exception:
        return False

if in_colab():
    from google.colab import drive
    drive.mount("/content/drive")
    REPO_PATH = Path("/content/drive/My Drive/einstein_program_synthesis/scripts")  # adjust if needed
else:
    REPO_PATH = Path.home() / "Documents" / "Code" / "einstein_program_synthesis" / "scripts"  # adjust if needed

sys.path.insert(0, str(REPO_PATH.resolve()))
print("using repo path:", REPO_PATH.resolve())
print("exists?", REPO_PATH.exists())
print("contains:", [p.name for p in REPO_PATH.glob("*.py")])


Mounted at /content/drive
using repo path: /content/drive/My Drive/einstein_program_synthesis/scripts
exists? True
contains: ['context.py', 'judgments.py', 'einstein_types.py', 'synthesis_state.py', 'synthesis_primitives.py', 'typing_rules.py', 'simplify.py', 'pure_bayes_search.py', 'enumeration_search.py', 'bayes_neural_search.py']


# Bayes neural search

In [2]:
import importlib
import bayes_neural_search as bn
from einstein_types import Art, Emp

bn = importlib.reload(bn)
print("module path:", bn.__file__)

module path: /content/drive/My Drive/einstein_program_synthesis/scripts/bayes_neural_search.py


In [3]:
import sys, torch, numpy as np
print(sys.version)
print("torch:", torch.__version__, " numpy:", np.__version__)

3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
torch: 2.6.0+cu124  numpy: 2.0.2


## Search settings

In [4]:
bn.MAX_SAMPLES_BAYES = 100
bn.BATCH_SIZE        = 64
bn.EPOCHS            = 3
bn.LR                = 1e-3

bn.MAX_SAMPLES       = 100
bn.CHAIN_LENGTH      = 8
bn.FEATURE_DIM       = 128

bn.TERM_WEIGHT       = 0.7
bn.TYPE_WEIGHT       = 0.3
bn.GOAL_TERM         = "t - u*x/c**2"
bn.GOAL_TYPE         = Art & Emp
bn.TARGET_TERM       = "f(-k*y + t*w - k*u*x/c)"

bn.MIX_BASE          = 0.1

bn.BACKWARD_NAMES               = {"BWD"}
bn.ENABLE_NO_REPEAT             = True # False
bn.ENABLE_SUB1_CONSTRAINT       = True # False
bn.ENABLE_SIM_SIMPUS_CONSTRAINT = True # False
bn.ENABLE_SIMPUS_SIM_CONSTRAINT = True # False
bn.ENABLE_BACKWARD_GATE         = True # False
bn.ENABLE_BACKWARD_ONLY_MODE    = False

## Neural network

In [5]:
net = bn.train_policy()

/content/drive/My Drive/einstein_program_synthesis/scripts/bayes_neural_search.py:194: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  return torch.tensor(S), torch.tensor(A)


Epoch  0  policy_loss=1.6905
Epoch  1  policy_loss=1.6429
Epoch  2  policy_loss=1.6041


## Search

In [ ]:
results = bn.run_one_experiment_paths(net, max_samples=bn.MAX_SAMPLES, print_trace=1)

Program 1: SUB1 + SIM+ + SIM  →  t - u*x/c**2 : Art
Program 2: FWD + SIM+ + SIM + SUB1 + BWD + BWD  →  f(-k*y + t*w - u*w*x/c**2) : Art
Program 3: SUB1 + FWD + SIM+ + SIM  →  f(-k*y + t*w - u*w*x/c**2) : Art
Program 4: SUB1 + SIM+ + FWD + SIM  →  f(-k*y + w*(t - u*x/c**2)) : Art
Program 5: SIM+ + FWD + SUB1 + SIM  →  f(c*k*(t - u*x/c**2) - k*y) : Art
Program 6: FWD + SUB1 + SIM + SIM+  →  f(k*(c**2*t - c*y - u*x)/c) : Art
Program 7: SIM+ + SUB1 + SIM  →  t - u*x/c**2 : Art
Program 8: SIM + SUB1  →  t - u*x/c**2 : Art
Program 9: FWD + SUB1 + SIM+ + SIM  →  f(k*(c**2*t - c*y - u*x)/c) : Art
Program 10: SIM + SUB1 + SIM+  →  t - u*x/c**2 : Art
Program 11: FWD + SUB1 + SIM+  →  f(k*(c**2*t - c*y - u*x)/c) : Art
Program 12: FWD + SIM + SIM+ + SUB1 + CHG + BWD  →  f(-k*y + t*w - k*u*x/c) : (Art ∧ Emp)
Program 13: SIM  →  t - u*x/c**2 : Art
Program 14: SIM+ + SIM + SUB1  →  t - u*x/c**2 : Art
Program 15: SUB1 + SIM + SIM+  →  t - u*x/c**2 : Art
Program 16: FWD + SIM+ + SUB1 + BWD + BWD + SIM 

In [7]:
results

93